In [1]:
using JuMP, HiGHS
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
using FileIO
using PlotlyBase
using PlotlyKaleido
ENV["COLUMNS"]=120;

In [2]:
datadir = joinpath("ed_data") 
# Note: joinpath is a good way to create path reference that is agnostic
# to what file system you are using (e.g. whether directories are denoted 
# with a forward or backwards slash).
gen_info = CSV.read(joinpath(datadir,"Generators_data.csv"), DataFrame);
fuels = CSV.read(joinpath(datadir,"Fuels_data.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Demand.csv"), DataFrame);
gen_variable = CSV.read(joinpath(datadir,"Generators_variability.csv"), DataFrame);


In [3]:
# Rename all columns to lowercase (by convention)
for f in [gen_info, fuels, loads, gen_variable]
    rename!(f,lowercase.(names(f)))
end
     

In [4]:
# Keep only the columns relevant to our ED model 
# (We'll come back to other columns in our unit commitment notebooks)
select!(gen_info, 1:26, :stor) 
gen_df = outerjoin(gen_info,  fuels, on = :fuel) # load in fuel costs and add to data frame
rename!(gen_df, :cost_per_mmbtu => :fuel_cost)   # rename column for fuel cost
gen_df[ismissing.(gen_df[:,:fuel_cost]), :fuel_cost] .= 0

0-element view(::Vector{Union{Missing, Float64}}, Int64[]) with eltype Union{Missing, Float64}

In [5]:

# create "is_variable" column to indicate if this is a variable generation source (e.g. wind, solar)
# Note: Julia's strict vectorization syntax requires us to create a new variable using the '!' indexing operator
# and the '.=' broadcasting assignment. See: https://julia.guide/broadcasting
gen_df[!, :is_variable] .= false
gen_df[in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(gen_df.resource),
    :is_variable] .= true;

In [6]:

# create full name of generator (including geographic location and cluster number)
#  for use with variable generation dataframe
gen_df.gen_full = lowercase.(gen_df.region .* "_" .* gen_df.resource .* "_" .* string.(gen_df.cluster) .* ".0");

# remove generators with no capacity (e.g. new build options that we'd use if this was capacity expansion problem)
gen_df = gen_df[gen_df.existing_cap_mw .> 0,:];


In [7]:
df = gen_df[1:5,1:5]

Row,r_id,resource,region,existing_cap_mw,num_units
,Int64?,String?,String15?,Float64?,Int64?
1,1,biomass,WEC_SDGE,21.4,20
2,2,hydroelectric_pumped_storage,WEC_SDGE,42.0,2
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,607.1,1
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,30.0,1
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,49.4,1


In [8]:
df = copy(gen_df)
select!(df, 1:5)
df = df[1:5,:]

Row,r_id,resource,region,existing_cap_mw,num_units
,Int64?,String?,String15?,Float64?,Int64?
1,1,biomass,WEC_SDGE,21.4,20
2,2,hydroelectric_pumped_storage,WEC_SDGE,42.0,2
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,607.1,1
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,30.0,1
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,49.4,1


In [9]:

stack(df, 
    [:existing_cap_mw, :num_units], 
    variable_name=:var,
    value_name=:val)
     

Row,r_id,resource,region,var,val
,Int64?,String?,String15?,String,Float64?
1,1,biomass,WEC_SDGE,existing_cap_mw,21.4
2,2,hydroelectric_pumped_storage,WEC_SDGE,existing_cap_mw,42.0
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,607.1
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,30.0
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,49.4
6,1,biomass,WEC_SDGE,num_units,20.0
7,2,hydroelectric_pumped_storage,WEC_SDGE,num_units,2.0
8,3,natural_gas_fired_combined_cycle,WEC_SDGE,num_units,1.0
9,4,natural_gas_fired_combined_cycle,WEC_SDGE,num_units,1.0


In [10]:
gen_variable.hour = mod.(gen_variable.hour .- 9, 24) .+ 1 
sort!(gen_variable, :hour)
loads.hour = mod.(loads.hour .- 9, 24) .+ 1
sort!(loads, :hour);
describe(gen_variable)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,hour,12.5,1,12.5,24,0,Int64
2,wec_sdge_biomass_1.0,1.0,1,1.0,1,0,Int64
3,wec_sdge_hydroelectric_pumped_storage_1.0,1.0,1,1.0,1,0,Int64
4,wec_sdge_natural_gas_fired_combined_cycle_1.0,1.0,1,1.0,1,0,Int64
5,wec_sdge_natural_gas_fired_combined_cycle_2.0,1.0,1,1.0,1,0,Int64
6,wec_sdge_natural_gas_fired_combined_cycle_3.0,1.0,1,1.0,1,0,Int64
7,wec_sdge_natural_gas_fired_combined_cycle_4.0,1.0,1,1.0,1,0,Int64
8,wec_sdge_natural_gas_fired_combustion_turbine_1.0,1.0,1,1.0,1,0,Int64
9,wec_sdge_natural_gas_fired_combustion_turbine_2.0,1.0,1,1.0,1,0,Int64


In [11]:
gen_variable_long = stack(gen_variable, 
                        Not(:hour), 
                        variable_name=:gen_full,
                        value_name=:cf);
# Now we have a "long" dataframe; 
# let's look at the first 6 entries of a wind resource for example
first(gen_variable_long[gen_variable_long.gen_full.=="wec_sdge_onshore_wind_turbine_1.0",:],6)

Row,hour,gen_full,cf
,Int64,String,Float64
1,1,wec_sdge_onshore_wind_turbine_1.0,0.1694
2,1,wec_sdge_onshore_wind_turbine_1.0,0.3336
3,1,wec_sdge_onshore_wind_turbine_1.0,0.0812
4,1,wec_sdge_onshore_wind_turbine_1.0,0.177
5,1,wec_sdge_onshore_wind_turbine_1.0,0.0923
6,1,wec_sdge_onshore_wind_turbine_1.0,0.0412


In [12]:

hr = 24  # pick 4pm on a spring day
loads_single = loads[loads[:,:hour] .== hr, Not(:hour)];
var_cf_single = gen_variable_long[
    gen_variable_long.hour .== hr, 
    Not(:hour)]
     

Row,gen_full,cf
,String,Float64
1,wec_sdge_biomass_1.0,1.0
2,wec_sdge_biomass_1.0,1.0
3,wec_sdge_biomass_1.0,1.0
4,wec_sdge_biomass_1.0,1.0
5,wec_sdge_biomass_1.0,1.0
6,wec_sdge_biomass_1.0,1.0
7,wec_sdge_biomass_1.0,1.0
8,wec_sdge_biomass_1.0,1.0
9,wec_sdge_biomass_1.0,1.0


In [13]:
#=
Function to solve economic dispatch problem (single-time period, single-zone)
Inputs:
    gen_df -- dataframe with generator info
    loads  -- dataframe with load info
    gen_variable -- capacity factors of variable generators (in "long" format)
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function economic_dispatch_single(gen_df, loads, gen_variable)
    ED = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # A set of all variable generators
    G_var = gen_df[gen_df[!,:is_variable] .== 1,:r_id] 
      # A set of all non-variable generators
    G_nonvar = gen_df[gen_df[!,:is_variable] .== 0,:r_id]
      # Set of all generators
    G = gen_df.r_id
    # Extract some parameters given the input data
      # Generator capacity factor time series for variable generators
    gen_var_cf = innerjoin(gen_variable, 
                    gen_df[gen_df.is_variable .== 1 , 
                        [:r_id, :gen_full, :existing_cap_mw]], 
                    on = :gen_full)
        
    # Decision variables   
    @variables(ED, begin
        GEN[G]  >= 0     # generation
        # Note: we assume Pmin = 0 for all resources for simplicty here
    end)
                
    # Objective function
    @objective(ED, Min, 
        sum( (gen_df[i,:heat_rate_mmbtu_per_mwh] * gen_df[i,:fuel_cost] +
            gen_df[i,:var_om_cost_per_mwh]) * GEN[i] 
                        for i in G_nonvar) + 
        sum(gen_df[i,:var_om_cost_per_mwh] * GEN[i] 
                        for i in G_var)
    )

    # Demand constraint
    @constraint(ED, cDemand, 
        sum(GEN[i] for i in G) == loads[1,:demand])

    # Capacity constraint (non-variable generation)
    for i in G_nonvar
        @constraint(ED, GEN[i] <= gen_df[i,:existing_cap_mw])
    end

    # Variable generation capacity constraint
    for i in 1:nrow(gen_var_cf)
        @constraint(ED, GEN[gen_var_cf[i,:r_id] ] <= 
                        gen_var_cf[i,:cf] *
                        gen_var_cf[i,:existing_cap_mw])
    end

    # Solve statement (! indicates runs in place)
    optimize!(ED)

    # Dataframe of optimal decision variables
    solution = DataFrame(
        r_id = gen_df.r_id,
        resource = gen_df.resource,
        gen = value.(GEN).data
        )

    # Return the solution and objective as named tuple
    return (
        solution = solution, 
        cost = objective_value(ED),
    )
end
     

economic_dispatch_single (generic function with 1 method)

In [21]:
gen_df

Row,r_id,resource,region,existing_cap_mw,num_units,cap_size,var_om_cost_per_mwh,var_om_cost_per_mwh_in,fuel,heat_rate_mmbtu_per_mwh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,min_power,ramp_up_percentage,ramp_dn_percentage,start_cost_per_mw,start_fuel_mmbtu_per_mw,up_time,down_time,self_disch,eff_up,eff_down,ratio_power_to_energy,min_duration,max_duration,cluster,stor,fuel_cost,co2_content_tons_per_mmbtu,is_variable,gen_full
,Int64?,String?,String15?,Float64?,Int64?,Float64?,Float64?,Int64?,String31,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64?,Float64?,Int64?,Int64?,Int64?,Float64?,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Float64?,Float64?,Bool,String
1,1,biomass,WEC_SDGE,21.4,20,1.07,5.234,0,None,12.76,1.471,4.147,0.71,1.0,1.0,0,0.0,0,0,0,1.0,1.0,1.0,0,0,1,0,0.0,0.0,false,wec_sdge_biomass_1.0
2,2,hydroelectric_pumped_storage,WEC_SDGE,42.0,2,21.0,0.0,0,None,0.0,0.0,0.0,0.0,1.0,1.0,0,0.0,0,0,0,0.866,0.866,0.0063,0,0,1,1,0.0,0.0,false,wec_sdge_hydroelectric_pumped_storage_1.0
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,607.1,1,607.1,3.4,0,pacific_naturalgas,7.52,0.0,0.0,0.362,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,1,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_1.0
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,30.0,1,30.0,4.3,0,pacific_naturalgas,10.05,0.0,0.0,0.55,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,2,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_2.0
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,49.4,1,49.4,4.3,0,pacific_naturalgas,9.73,0.0,0.0,0.5,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,3,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_3.0
6,6,natural_gas_fired_combined_cycle,WEC_SDGE,570.0,1,570.0,3.4,0,pacific_naturalgas,7.15,0.0,0.0,0.351,0.4,0.4,87,2.0,6,6,0,1.0,1.0,1.0,0,0,4,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_4.0
7,7,natural_gas_fired_combined_cycle,WEC_SDGE,336.0,3,112.0,10.8,0,pacific_naturalgas,10.03,0.0,0.0,0.446,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,5,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_5.0
8,8,natural_gas_fired_combined_cycle,WEC_SDGE,336.0,3,112.0,10.8,0,pacific_naturalgas,10.1,0.0,0.0,0.446,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,6,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combined_cycle_6.0
9,9,natural_gas_fired_combustion_turbine,WEC_SDGE,91.6,2,45.8,10.8,0,pacific_naturalgas,10.69,0.0,0.0,0.8,3.78,3.78,113,3.5,1,1,0,1.0,1.0,1.0,0,0,1,0,2.57,0.05306,false,wec_sdge_natural_gas_fired_combustion_turbine_1.0


In [15]:
loads_single

Row,demand
,Int64
1,1715
2,1617
3,1728
4,1725
5,1757
6,1759
7,1782
8,1672
9,1556


In [16]:
var_cf_single

Row,gen_full,cf
,String,Float64
1,wec_sdge_biomass_1.0,1.0
2,wec_sdge_biomass_1.0,1.0
3,wec_sdge_biomass_1.0,1.0
4,wec_sdge_biomass_1.0,1.0
5,wec_sdge_biomass_1.0,1.0
6,wec_sdge_biomass_1.0,1.0
7,wec_sdge_biomass_1.0,1.0
8,wec_sdge_biomass_1.0,1.0
9,wec_sdge_biomass_1.0,1.0


In [17]:
solution = economic_dispatch_single(gen_df, loads_single, var_cf_single);
solution.solution

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
1 rows, 23 cols, 23 nonzeros
1 rows, 22 cols, 22 nonzeros
Presolve : Reductions: rows 1(-1117); columns 22(-3); elements 22(-1120)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1(1715) 0s
          1     4.3076755109e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  4.3076755109e+04
HiGHS run time      :          0.00


Row,r_id,resource,gen
,Int64?,String?,Float64
1,1,biomass,21.4
2,2,hydroelectric_pumped_storage,42.0
3,3,natural_gas_fired_combined_cycle,607.1
4,4,natural_gas_fired_combined_cycle,30.0
5,5,natural_gas_fired_combined_cycle,49.4
6,6,natural_gas_fired_combined_cycle,570.0
7,7,natural_gas_fired_combined_cycle,336.0
8,8,natural_gas_fired_combined_cycle,57.5138
9,9,natural_gas_fired_combustion_turbine,0.0


In [18]:
supply_curve = leftjoin(gen_df,
                var_cf_single, 
                on = :gen_full)
supply_curve[!, :varcost] .= 0.0
supply_curve[!, :cap] .= 0.0

# Store varcost of non-variable generators:
I = supply_curve[!,:is_variable] .== 0  # `I` contains indexes to the non-variable generators
supply_curve[I,:varcost] .= 
    supply_curve[I,:heat_rate_mmbtu_per_mwh] .* 
        supply_curve[I,:fuel_cost] .+
    supply_curve[I,:var_om_cost_per_mwh]

# Calculate available capacity for each generator
supply_curve[I,:cap] = supply_curve[I,:existing_cap_mw]

# Store varcost of variable generators (in this case, 0)
I = (supply_curve[:,:is_variable] .== 1) # `I` contains indexes to the variable generators
supply_curve[I,:varcost] = supply_curve[I,:var_om_cost_per_mwh]

# Calculate available capacity for each generator
# (adjusted for variable generation)
supply_curve[I,:cap] = supply_curve[I,:existing_cap_mw] .* supply_curve[I,:cf]

sort!(supply_curve, :varcost);

In [19]:
rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

rectangle (generic function with 1 method)